In [4]:
from langchain.prompts import PromptTemplate# type: ignore

# Experiment with prompt structure

In [5]:
import csv

# Specify the path to your CSV file
csv_file_path = "student/Student Data Rows - Sheet1.csv"

# Initialize an empty list to store the data
data = []

# Open the CSV file and read its contents
with open(csv_file_path, "r") as file:
    reader = csv.reader(file)
    # Skip the header row if it exists
    next(reader, None)
    # Iterate over each row in the CSV file
    for row in reader:
        data.append(row)

# Print the loaded data (optional)
print(data)


[['John Doe', 'Science', '10', 'Mathematics;Physics;Chemistry', 'Visual;Auditory;Kinesthetic', 'Score high in exams;Understand concepts deeply', 'Time management;Complexity of subjects', 'Chess Club;Debate Team'], ['Jane Smith', 'Arts', '11', 'English Literature;History;Art and Design', 'Visual;Verbal;Interpersonal', 'Improve writing skills;Learn about historical events', 'Public speaking;Meeting deadlines', 'Basketball Team;Drama Club'], ['Michael Johnson', 'Commerce', '12', 'Accounting;Business Studies;Economics', 'Logical;Social;Kinesthetic', 'Achieve financial literacy;Gain business acumen', 'Managing finances;Understanding market trends', 'Entrepreneurship Club;Financial Literacy Workshop'], ['Maria Garcia', 'STEM', '9', 'Biology;Computer Science;Robotics', 'Logical;Visual;Hands-on', 'Explore STEM careers;Build technical skills', 'Time management;Complexity of coding tasks', 'Science Club;Robotics Competition'], ['Ahmed Khan', 'Humanities', '10', 'Sociology;Psychology;Political Sc

In [8]:
data[0]

['John Doe',
 'Science',
 '10',
 'Mathematics;Physics;Chemistry',
 'Visual;Auditory;Kinesthetic',
 'Score high in exams;Understand concepts deeply',
 'Time management;Complexity of subjects',
 'Chess Club;Debate Team']

In [14]:
i = 2

Name  = data[i][0]
Grade = data[i][1]
Field =data[i][2]
Subjects=data[i][3]
Learning_Styles=data[i][4]
Extracurricular=data[i][5]
Objectives=data[i][6]
Challenges=data[i][7]

In [21]:
prompt = f""""
Given the following information about a student, create a detailed and personalized study plan that addresses their academic and personal needs:

1. Student Profile:
   - Name: {Name}
   - Grade Level: {Grade}
   - Field of Study: {Field}
   - Academic Subjects: {Subjects}
  

2. Preferred Learning Styles:
   {Learning_Styles}

3. Extracurricular Activities:
   {Extracurricular}

4. Personal Objectives:
   {Objectives}

5. Challenges
   {Challenges}
```

Based on the above information, generate a comprehensive study plan by adhering to these instructions:

1- Weekly Study Schedule: 
- Analyze student's Academic Subjects, and Field of Study.
- Develop a timetable that balances study time across subjects with the student’s 
extracurricular activities
- Ensure adequate time is allocated to each subject, especially those needing improvement.

2- Learning Strategies:
- Examine student's Preferred Learning Styles and Challenges.
- Explore and Recommend a specific study method for the student and resources tailored to the 
student’s learning styles and subjects. 
- Within the recommended study method, include suggestions for overcoming their specified Challenges.

3- Short-term and Long-term Goals: 
- Identify Student's Personal Objectives
- Explore relevance of their Personal Objectives with their Field of Study
- Define clear, achievable set of goals for the student’s academic journey, 
aligned with their Personal Objectives.

4- Assessment and Adjustment: 
- Develop a method for tracking the student’s progress based on their Weekly Study Schedule
- Ensure ongoing effectiveness and alignment with the student’s evolving needs and goals.

Ensure that the study plan is realistic, sustainable, and conducive to the student’s overall well-being and academic growth.

"""

In [25]:
print(prompt) #Experimentation of prompt itself of Deep learning AI

"
Given the following information about a student, create a detailed and personalized study plan that addresses their academic and personal needs:

1. Student Profile:
   - Name: Michael Johnson
   - Grade Level: Commerce
   - Field of Study: 12
   - Academic Subjects: Accounting;Business Studies;Economics
  

2. Preferred Learning Styles:
   Logical;Social;Kinesthetic

3. Extracurricular Activities:
   Achieve financial literacy;Gain business acumen

4. Personal Objectives:
   Managing finances;Understanding market trends

5. Challenges
   Entrepreneurship Club;Financial Literacy Workshop
```

Based on the above information, generate a comprehensive by adhering to these instructions:

1- Weekly Study Schedule: 
- Analyze student's Academic Subjects, and Field of Study.
- Develop a timetable that balances study time across subjects with the student’s 
extracurricular activities
- Ensure adequate time is allocated to each subject, especially those needing improvement.

2- Learning Strat

# Prompt Creation

In [6]:
class StudyPlanGenPrompt(PromptTemplate):
    @staticmethod
    def get_template(Name=None, Grade=None, Field=None, Subjects=None, Learning_Styles=None, Extracurricular=None, Objectives=None, Challenges=None):
        '''Returns prompt for generating detailed study plan'''
        task_template =  PromptTemplate.from_file('student/student.jinja', input_variables=['Name', 'Grade','Field', 'Subjects', 'Learning_Styles', 'Extracurricular', 'Objectives', 'Challenges'], template_format="jinja2")
        return task_template.format(Name=Name, Grade=Grade, Field=Field, Subjects=Subjects, Learning_Styles=Learning_Styles, Extracurricular=Extracurricular, Objectives=Objectives, Challenges=Challenges)

In [7]:
from openai import OpenAI
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY") or None
class StudyPlanGen():
    @staticmethod
    def generate_thread_title(Name, Grade, Field, Subjects, Learning_Styles, Extracurricular, Objectives, Challenges, deployment):
        chatgpt_deployment = deployment #! getModelDeployment in views.py
        #query = "Generate Title"
        prompt = StudyPlanGenPrompt.get_template(Name=Name, Grade=Grade, Field=Field, Subjects=Subjects, Learning_Styles=Learning_Styles, Extracurricular=Extracurricular, Objectives=Objectives, Challenges=Challenges) 
        try:
            client = OpenAI(
                api_key = OPENAI_API_KEY
            )
            completion = client.chat.completions.create(
                model = chatgpt_deployment,
                messages=[{"role":"system","content":prompt},{"role": "user", "content": "Develop a detailed and personalized study plan"}],
                temperature=0.7
            )
            
            return completion.choices[0].message.content
        except Exception as e:
            return "New Thread"


In [8]:
i = 2 #vary this number between 0 and len -1 of data for different students

Name  = data[i][0]
Grade = data[i][1]
Field =data[i][2]
Subjects=data[i][3]
Learning_Styles=data[i][4]
Extracurricular=data[i][5]
Objectives=data[i][6]
Challenges=data[i][7]

In [11]:
study_plan = StudyPlanGen.generate_thread_title(Name, Grade, Field, Subjects, Learning_Styles, Extracurricular, Objectives, Challenges, "gpt-3.5-turbo-1106")

In [12]:
# from pprint import pprint 
print(study_plan)

**1- Weekly Study Schedule:**
- Monday: 
  - 3:00 PM - 5:00 PM: Accounting
  - 6:00 PM - 7:30 PM: Business Studies
- Tuesday:
  - 3:00 PM - 5:00 PM: Economics
  - 6:00 PM - 7:30 PM: Achieve financial literacy (Extracurricular activity)
- Wednesday:
  - 3:00 PM - 5:00 PM: Business Studies
  - 6:00 PM - 7:30 PM: Accounting
- Thursday:
  - 3:00 PM - 5:00 PM: Economics
  - 6:00 PM - 7:30 PM: Entrepreneurship Club (Challenges)
- Friday:
  - 3:00 PM - 5:00 PM: Accounting
  - 6:00 PM - 7:30 PM: Financial Literacy Workshop (Challenges)
- Saturday:
  - 10:00 AM - 12:00 PM: Economics
  - 1:00 PM - 3:00 PM: Business Studies
- Sunday:
  - 10:00 AM - 12:00 PM: Review and catch up on any subjects as needed

**2- Learning Strategies:**
Michael's preferred learning styles are logical, social, and kinesthetic. To accommodate these styles and address his challenges, a recommended study method for him would be:
- Logical: Utilize textbooks, online resources, and practice problems to engage his logical th